In [27]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache
from langchain.schema import BaseOutputParser
from langchain.prompts import PromptTemplate, ChatPromptTemplate

set_llm_cache(SQLiteCache("cache.db"))

chat = ChatOpenAI(
    temperature=0.1
)

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))

# 리스트를 쉼표로 구분된 문자열로 변환하는 함수
def convert_list_to_string(activity_list):
    return ', '.join(activity_list)


template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an expert in packing for international travel. You recommend the luggage people need when traveling abroad with airplane. And You are a list of max {max_items} generating machine. Everything you are asked will be answered with a comma separated list in lowercase.Do NOT reply with anything else. And you only reply in Korean."),
        #("ai", "Ciao, mi chiamo {name}!"),
        (
            "human",
            "I am a {gender} and I am going to travel abroad to {destination}. \
            I plan to depart on {start_month} {start_day}th and stay for {days} days. I plan to mainly {activities} during the trip. What luggage should I pack? Could you please tell me in as much detail as possible?",
        ),
    ]
)


activities_list = ["수영", "하이킹", "관광"]

input_data = {
    "gender": "man",
    "destination": "Seoul",
    "start_month": "5",
    "start_day": "10",
    "days": "5",
    "activities": convert_list_to_string(activities_list),
    "max_items": 100
}

chain = template | chat | CommaOutputParser()
# Invoke the chain with the input
output = chain.invoke(input_data)

# Print the output
print(output)



['양말', '속옷', '반팔 티셔츠', '반바지', '수영복', '슬리퍼', '모자', '선글라스', '스마트폰', '충전기', '여권', '현지 통화', '우산', '스니커즈', '약품', '세면도구', '카메라', '지도', '물병']
